In [ ]:
#该代码是画NCEP图的。用了NCEP的各层风速数据和降水数据，请自行阅读

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.mpl.ticker as cticker
import cartopy.io.shapereader as shpreader
import numpy as np
import math
import xarray as xr
import pandas as pd
import scipy.stats
import datetime

In [ ]:
def monthdate(yearlist,season):
    resultlist=[]
    yearlist = map(lambda x: int(x), yearlist)
    if season=='autumn':
        for year in yearlist:
            for month in [9,10,11]:
                resultlist.append(datetime.date(year,month,1).isoformat())
    elif season=='winter':
        for year in yearlist:
            for month in [12]:
                resultlist.append(datetime.date(year,month,1).isoformat())
            for month in [1,2]:
                resultlist.append(datetime.date(year+1,month,1).isoformat())
    elif season=='spring':
        for year in yearlist:
            for month in [3,4,5]:
                resultlist.append(datetime.date(year,month,1).isoformat())
    elif season=='summer':
        for year in yearlist:
            for month in [6,7,8]:
                resultlist.append(datetime.date(year,month,1).isoformat())
    elif season=='overall':
        for year in yearlist:
            for month in list(range(3,13)):
                resultlist.append(datetime.date(year,month,1).isoformat())
            for month in list(range(1,3)):
                resultlist.append(datetime.date(year+1,month,1).isoformat())  
    return resultlist

def plusoneyear(yearlist):
    resultlist=[]
    for year in yearlist:
        resultlist.append(str(int(year)+1))
    return resultlist

#pEvent=['1982', '1983', '1987','1991','1992','1997','2002','2009','2014']#ENSO事件 #暂时去掉1979年前，
#nEvent=['1984','1985','1988','1995','1998','1999','2007','2010']#ENSO事件

pEvent=['1982','1983','1994','1997','2006','2012','2015','2018','2019'] #IOD事件，参考IOD事件的，并讲1979年后删去
nEvent=['1989','1992','1993','1996','1998','2010','2016']#IOD事件，参考IOD事件的，并讲1979年后删去

#pEvent=['1951','1953','1957','1963', '1965', '1969', '1972', '1976', '1982', '1983', '1987','1991','1992','1997','2002','2009','2014'] #为了泛用性，这次全部换为Event，不再出现相关名词
#选择法则，一样使用日本气象局的划分方法，当且仅当该年夏季出现了ENSO时，才会加入该年份，其他时间发生则不加入。现已更新为最新，https://ds.data.jma.go.jp/tcc/tcc/products/elnino/ensoevents.html，后几个ENSO事件都没覆盖到夏季
#nEvent=['1949','1950','1954','1955','1964','1970','1971','1973','1975','1984','1985','1988','1995','1998','1999','2007','2010']

Allyear = range(1979,2020)
Normalyear=[]
for year in Allyear:
    year=str(year)
    if year not in pEvent and year not in nEvent:
        Normalyear.append(year)
        
Allyearforcal=list(Allyear)

NORM_AUTUMN_DATE=monthdate(yearlist=Normalyear,season='autumn')
NORM_WINTER_DATE=monthdate(yearlist=Normalyear,season='winter')
NORM_SPRING_DATE=monthdate(yearlist=Normalyear,season='spring')
NORM_SUMMER_DATE=monthdate(yearlist=Normalyear,season='summer')

pEvent_AUTUMN_DATE=monthdate(yearlist=pEvent,season='autumn')
pEvent_WINTER_DATE=monthdate(yearlist=pEvent,season='winter')
pEvent_SPRING_DATE=monthdate(yearlist=pEvent,season='spring')
pEvent_SUMMER_DATE=monthdate(yearlist=pEvent,season='summer')

nEvent_AUTUMN_DATE=monthdate(yearlist=nEvent,season='autumn')
nEvent_WINTER_DATE=monthdate(yearlist=nEvent,season='winter')
nEvent_SPRING_DATE=monthdate(yearlist=nEvent,season='spring')
nEvent_SUMMER_DATE=monthdate(yearlist=nEvent,season='summer')

pEvent_OVERALL_DATE=monthdate(yearlist=pEvent,season='overall')
nEvent_OVERALL_DATE=monthdate(yearlist=nEvent,season='overall')
NORM_OVERALL_DATE=monthdate(yearlist=Normalyear,season='overall')

ALLYEAR_AUTUMN_DATE=monthdate(yearlist=Allyearforcal,season='autumn')
ALLYEAR_SUMMER_DATE=monthdate(yearlist=Allyearforcal,season='summer')

In [ ]:
uwinddataset = xr.open_dataset(r'uwnd.mon.mean.multpres.nc') 
vwinddataset = xr.open_dataset(r'vwnd.mon.mean.multpres.nc')
uwinddataset = uwinddataset.sel(level=850.0)
vwinddataset = vwinddataset.sel(level=850.0)

lon=uwinddataset['lon'].values
lat=uwinddataset['lat'].values

def meanwindvelocity(data,value,date): 
    value=str(value)
    for month in date:
        if month==date[0]:
            combinedata=data[value].loc[month]
        else:
            combinedata=combinedata+data[value].loc[month]
    combinedata=combinedata/len(date)
    return combinedata.values

pEvent_AUT_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=pEvent_AUTUMN_DATE)
pEvent_AUT_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=pEvent_AUTUMN_DATE)

nEvent_AUT_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=nEvent_AUTUMN_DATE)
nEvent_AUT_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=nEvent_AUTUMN_DATE)

NORM_AUT_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=NORM_AUTUMN_DATE)
NORM_AUT_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=NORM_AUTUMN_DATE)

AUT_p_NORM_U=pEvent_AUT_U-NORM_AUT_U
AUT_p_NORM_V=pEvent_AUT_V-NORM_AUT_V

AUT_n_NORM_U=nEvent_AUT_U-NORM_AUT_U
AUT_n_NORM_V=nEvent_AUT_V-NORM_AUT_V

AUT_p_n_U=pEvent_AUT_U-nEvent_AUT_U
AUT_p_n_V=pEvent_AUT_V-nEvent_AUT_V

pEvent_WIN_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=pEvent_WINTER_DATE)
pEvent_WIN_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=pEvent_WINTER_DATE)

nEvent_WIN_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=nEvent_WINTER_DATE)
nEvent_WIN_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=nEvent_WINTER_DATE)

NORM_WIN_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=NORM_WINTER_DATE)
NORM_WIN_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=NORM_WINTER_DATE)

WIN_p_NORM_U=pEvent_WIN_U-NORM_WIN_U
WIN_p_NORM_V=pEvent_WIN_V-NORM_WIN_V

WIN_n_NORM_U=nEvent_WIN_U-NORM_WIN_U
WIN_n_NORM_V=nEvent_WIN_V-NORM_WIN_V

WIN_p_n_U=pEvent_WIN_U-nEvent_WIN_U
WIN_p_n_V=pEvent_WIN_V-nEvent_WIN_V

pEvent_SPR_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=pEvent_SPRING_DATE)
pEvent_SPR_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=pEvent_SPRING_DATE)

nEvent_SPR_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=nEvent_SPRING_DATE)
nEvent_SPR_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=nEvent_SPRING_DATE)

NORM_SPR_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=NORM_SPRING_DATE)
NORM_SPR_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=NORM_SPRING_DATE)

SPR_p_NORM_U=pEvent_SPR_U-NORM_SPR_U
SPR_p_NORM_V=pEvent_SPR_V-NORM_SPR_V

SPR_n_NORM_U=nEvent_SPR_U-NORM_SPR_U
SPR_n_NORM_V=nEvent_SPR_V-NORM_SPR_V

SPR_p_n_U=pEvent_SPR_U-nEvent_SPR_U
SPR_p_n_V=pEvent_SPR_V-nEvent_SPR_V

pEvent_SUM_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=pEvent_SUMMER_DATE)
pEvent_SUM_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=pEvent_SUMMER_DATE)

nEvent_SUM_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=nEvent_SUMMER_DATE)
nEvent_SUM_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=nEvent_SUMMER_DATE)

NORM_SUM_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=NORM_SUMMER_DATE)
NORM_SUM_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=NORM_SUMMER_DATE)

SUM_p_NORM_U=pEvent_SUM_U-NORM_SUM_U
SUM_p_NORM_V=pEvent_SUM_V-NORM_SUM_V

SUM_n_NORM_U=nEvent_SUM_U-NORM_SUM_U
SUM_n_NORM_V=nEvent_SUM_V-NORM_SUM_V

SUM_p_n_U=pEvent_SUM_U-nEvent_SUM_U
SUM_p_n_V=pEvent_SUM_V-nEvent_SUM_V

pEvent_ALL_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=pEvent_OVERALL_DATE)
pEvent_ALL_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=pEvent_OVERALL_DATE)

nEvent_ALL_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=nEvent_OVERALL_DATE)
nEvent_ALL_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=nEvent_OVERALL_DATE)

NORM_ALL_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=NORM_OVERALL_DATE)
NORM_ALL_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=NORM_OVERALL_DATE)

ALL_p_NORM_U=pEvent_ALL_U-NORM_ALL_U
ALL_p_NORM_V=pEvent_ALL_V-NORM_ALL_V

ALL_p_n_U=pEvent_ALL_U-nEvent_ALL_U
ALL_p_n_V=pEvent_ALL_V-nEvent_ALL_V

ALL_n_NORM_U=nEvent_ALL_U-NORM_ALL_U
ALL_n_NORM_V=nEvent_ALL_V-NORM_ALL_V

ALL_SUM_U=meanwindvelocity(data=uwinddataset,value='uwnd',date=ALLYEAR_SUMMER_DATE)
ALL_SUM_V=meanwindvelocity(data=vwinddataset,value='vwnd',date=ALLYEAR_SUMMER_DATE)

SUM_p_ALL_U=pEvent_SUM_U - ALL_SUM_U
SUM_p_ALL_V=pEvent_SUM_V - ALL_SUM_V

SUM_n_ALL_U=nEvent_SUM_U - ALL_SUM_U
SUM_n_ALL_V=nEvent_SUM_V - ALL_SUM_V

SUM_NORM_ALL_U=NORM_SUM_U - ALL_SUM_U
SUM_NORM_ALL_V=NORM_SUM_V - ALL_SUM_V

In [ ]:
precip = xr.open_dataset(r'precip.mon.mean.ehd.nc')
lonp=precip['lon'].values
latp=precip['lat'].values

def valuesget(data,value,date): 
    value=str(value)
    for month in date:
        if month==date[0]:
            combinedata=data[value].loc[month]
        else:
            combinedata=combinedata+data[value].loc[month]
    combinedata=combinedata/len(date)
    return combinedata.values

pEvent_AUT_P=valuesget(data=precip,value='precip',date=pEvent_AUTUMN_DATE)
nEvent_AUT_P=valuesget(data=precip,value='precip',date=nEvent_AUTUMN_DATE)
NORM_AUT_P=valuesget(data=precip,value='precip',date=NORM_AUTUMN_DATE)

AUT_p_NORM_P=pEvent_AUT_P-NORM_AUT_P  #Autumn  Postive-Normal  precipitation
AUT_p_n_P=pEvent_AUT_P-nEvent_AUT_P
AUT_n_NORM_P=nEvent_AUT_P-NORM_AUT_P 

pEvent_SPR_P=valuesget(data=precip,value='precip',date=pEvent_SPRING_DATE)
nEvent_SPR_P=valuesget(data=precip,value='precip',date=nEvent_SPRING_DATE)
NORM_SPR_P=valuesget(data=precip,value='precip',date=NORM_SPRING_DATE)

SPR_p_NORM_P=pEvent_SPR_P-NORM_SPR_P 
SPR_p_n_P=pEvent_SPR_P-nEvent_SPR_P
SPR_n_NORM_P=nEvent_SPR_P-NORM_SPR_P  

pEvent_SUM_P=valuesget(data=precip,value='precip',date=pEvent_SUMMER_DATE)
nEvent_SUM_P=valuesget(data=precip,value='precip',date=nEvent_SUMMER_DATE)
NORM_SUM_P=valuesget(data=precip,value='precip',date=NORM_SUMMER_DATE)

SUM_p_NORM_P=pEvent_SUM_P-NORM_SUM_P  
SUM_p_n_P=pEvent_SUM_P-nEvent_SUM_P
SUM_n_NORM_P=nEvent_SUM_P-NORM_SUM_P  

pEvent_WIN_P=valuesget(data=precip,value='precip',date=pEvent_WINTER_DATE)
nEvent_WIN_P=valuesget(data=precip,value='precip',date=nEvent_WINTER_DATE)
NORM_WIN_P=valuesget(data=precip,value='precip',date=NORM_WINTER_DATE)

WIN_p_NORM_P=pEvent_WIN_P-NORM_WIN_P  
WIN_p_n_P=pEvent_WIN_P-nEvent_WIN_P
WIN_n_NORM_P=nEvent_WIN_P-NORM_WIN_P  

pEvent_ALL_P=valuesget(data=precip,value='precip',date=pEvent_OVERALL_DATE)
nEvent_ALL_P=valuesget(data=precip,value='precip',date=nEvent_OVERALL_DATE)
NORM_ALL_P=valuesget(data=precip,value='precip',date=NORM_OVERALL_DATE)

ALL_p_NORM_P=pEvent_ALL_P-NORM_ALL_P 
ALL_p_n_P=pEvent_ALL_P-nEvent_ALL_P
ALL_n_NORM_P=nEvent_ALL_P-NORM_ALL_P

ALL_SUM_P=valuesget(data=precip,value='precip',date=ALLYEAR_SUMMER_DATE)
SUM_p_ALL_P=pEvent_SUM_P - ALL_SUM_P
SUM_n_ALL_P=pEvent_SUM_P - ALL_SUM_P
SUM_NORM_ALL_P=NORM_SUM_P - ALL_SUM_P

In [ ]:
#ENSO夏季异常
proj = ccrs.PlateCarree(central_longitude=0)
leftlon, rightlon, lowerlat, upperlat = (80,140,10,40)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
fig1 = plt.figure(figsize=(12,8))
f1_ax1 = fig1.add_axes([0.1, 0.1, 0.6, 0.6],projection = proj)
f1_ax1.set_extent([leftlon, rightlon, lowerlat, upperlat], crs=ccrs.PlateCarree())
f1_ax1.add_feature(cfeature.COASTLINE.with_scale('50m'))
f1_ax1.set_xticks(np.arange(leftlon,rightlon+10,10), crs=ccrs.PlateCarree())
f1_ax1.set_yticks(np.arange(lowerlat,upperlat+10,10), crs=ccrs.PlateCarree())
f1_ax1.xaxis.set_major_formatter(lon_formatter)
f1_ax1.yaxis.set_major_formatter(lat_formatter)



C = f1_ax1.contourf(lonp, latp,(SUM_p_n_P)*3,levels=[-12,-6,0,6,12,18],colors=['red','tomato','lightblue','royalblue','blue'], transform=ccrs.PlateCarree())
position=fig1.add_axes([0.72, 0.171, 0.02, 0.453])
fig1.colorbar(C,cax=position,orientation='vertical')
Q = f1_ax1.quiver(lon, lat, SUM_p_n_U,SUM_p_n_V,scale=30,minshaft=0.1, pivot='mid',width=0.0018,headwidth=4,transform=ccrs.PlateCarree())

plt.savefig(r"Figure\IOD\SUM_p_n_20210301.svg",bbox_inches='tight')

In [ ]:
#ENSO秋季异常
proj = ccrs.PlateCarree(central_longitude=0)
leftlon, rightlon, lowerlat, upperlat = (80,140,10,40)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
fig1 = plt.figure(figsize=(12,8))
f1_ax1 = fig1.add_axes([0.1, 0.1, 0.6, 0.6],projection = proj)
f1_ax1.set_extent([leftlon, rightlon, lowerlat, upperlat], crs=ccrs.PlateCarree())
f1_ax1.add_feature(cfeature.COASTLINE.with_scale('50m'))
f1_ax1.set_xticks(np.arange(leftlon,rightlon+10,10), crs=ccrs.PlateCarree())
f1_ax1.set_yticks(np.arange(lowerlat,upperlat+10,10), crs=ccrs.PlateCarree())
f1_ax1.xaxis.set_major_formatter(lon_formatter)
f1_ax1.yaxis.set_major_formatter(lat_formatter)



C = f1_ax1.contourf(lonp, latp,(AUT_p_n_P)*3,levels=[-12,-6,0,6,12,18],colors=['red','tomato','lightblue','royalblue','blue'], transform=ccrs.PlateCarree())
position=fig1.add_axes([0.72, 0.171, 0.02, 0.453])
fig1.colorbar(C,cax=position,orientation='vertical')
Q = f1_ax1.quiver(lon, lat, AUT_p_n_U,AUT_p_n_V,scale=30,minshaft=0.1, pivot='mid',width=0.0018,headwidth=4,transform=ccrs.PlateCarree())

R=f1_ax1.plot(112.823, 25.1863, 'o',color='red',  transform=ccrs.PlateCarree())

plt.savefig(r"Figure\IOD\AUT_p_n_20210301.svg",bbox_inches='tight')

In [ ]:
#ENSO冬季异常
proj = ccrs.PlateCarree(central_longitude=0)
leftlon, rightlon, lowerlat, upperlat = (80,140,10,40)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
fig1 = plt.figure(figsize=(12,8))
f1_ax1 = fig1.add_axes([0.1, 0.1, 0.6, 0.6],projection = proj)
f1_ax1.set_extent([leftlon, rightlon, lowerlat, upperlat], crs=ccrs.PlateCarree())
f1_ax1.add_feature(cfeature.COASTLINE.with_scale('50m'))
f1_ax1.set_xticks(np.arange(leftlon,rightlon+10,10), crs=ccrs.PlateCarree())
f1_ax1.set_yticks(np.arange(lowerlat,upperlat+10,10), crs=ccrs.PlateCarree())
f1_ax1.xaxis.set_major_formatter(lon_formatter)
f1_ax1.yaxis.set_major_formatter(lat_formatter)



C = f1_ax1.contourf(lonp, latp,WIN_p_n_P*3,levels=[-12,-6,0,6,12,18],colors=['red','tomato','lightblue','royalblue','blue'], transform=ccrs.PlateCarree())
position=fig1.add_axes([0.72, 0.171, 0.02, 0.453])
fig1.colorbar(C,cax=position,orientation='vertical')
Q = f1_ax1.quiver(lon, lat, WIN_p_n_U,WIN_p_n_V,scale=30,minshaft=0.1, pivot='mid',width=0.0018,headwidth=4,transform=ccrs.PlateCarree())
R=f1_ax1.plot(112.823, 25.1863, 'o',color='red',  transform=ccrs.PlateCarree())

plt.savefig(r"Figure\IOD\WIN_p_n_20210301.svg",bbox_inches='tight')

In [ ]:
#ENSO春季异常
proj = ccrs.PlateCarree(central_longitude=0)
leftlon, rightlon, lowerlat, upperlat = (80,140,10,40)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
fig1 = plt.figure(figsize=(12,8))
f1_ax1 = fig1.add_axes([0.1, 0.1, 0.6, 0.6],projection = proj)
f1_ax1.set_extent([leftlon, rightlon, lowerlat, upperlat], crs=ccrs.PlateCarree())
f1_ax1.add_feature(cfeature.COASTLINE.with_scale('50m'))
f1_ax1.set_xticks(np.arange(leftlon,rightlon+10,10), crs=ccrs.PlateCarree())
f1_ax1.set_yticks(np.arange(lowerlat,upperlat+10,10), crs=ccrs.PlateCarree())
f1_ax1.xaxis.set_major_formatter(lon_formatter)
f1_ax1.yaxis.set_major_formatter(lat_formatter)



C = f1_ax1.contourf(lonp, latp,SPR_p_n_P*3,levels=[-12,-6,0,6,12,18],colors=['red','tomato','lightblue','royalblue','blue'], transform=ccrs.PlateCarree())
position=fig1.add_axes([0.72, 0.171, 0.02, 0.453])
fig1.colorbar(C,cax=position,orientation='vertical')
Q = f1_ax1.quiver(lon, lat, SPR_p_n_U,SPR_p_n_V,scale=30,minshaft=0.1, pivot='mid',width=0.0018,headwidth=4,transform=ccrs.PlateCarree())
R=f1_ax1.plot(112.823, 25.1863, 'o',color='red',  transform=ccrs.PlateCarree())

plt.savefig(r"Figure\IOD\SPR_p_n_20210301.svg",bbox_inches='tight')

In [ ]:
proj = ccrs.PlateCarree(central_longitude=0)
leftlon, rightlon, lowerlat, upperlat = (80,140,10,40)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
fig1 = plt.figure(figsize=(12,8))
f1_ax1 = fig1.add_axes([0.1, 0.1, 0.6, 0.6],projection = proj)
f1_ax1.set_extent([leftlon, rightlon, lowerlat, upperlat], crs=ccrs.PlateCarree())

f1_ax1.set_xticks(np.arange(leftlon,rightlon+10,10), crs=ccrs.PlateCarree())
f1_ax1.set_yticks(np.arange(lowerlat,upperlat+10,10), crs=ccrs.PlateCarree())
f1_ax1.xaxis.set_major_formatter(lon_formatter)
f1_ax1.yaxis.set_major_formatter(lat_formatter)


Q = f1_ax1.quiver(np.array([125.0]),np.array([30.0]),np.array([2]),np.array([0]),scale=30,minshaft=0.1, pivot='mid',width=0.0018,headwidth=4,transform=ccrs.PlateCarree())
plt.text(124,30.5, r'$2m/s$',transform=ccrs.PlateCarree(), fontweight=100, fontsize=8)

plt.savefig(r"Figure\IOD\Legend_scale30_20210227.svg")

In [ ]:
#全年降水
proj = ccrs.PlateCarree(central_longitude=0)
leftlon, rightlon, lowerlat, upperlat = (80,140,10,40)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
fig1 = plt.figure(figsize=(12,8))
f1_ax1 = fig1.add_axes([0.1, 0.1, 0.6, 0.6],projection = proj)
f1_ax1.set_extent([leftlon, rightlon, lowerlat, upperlat], crs=ccrs.PlateCarree())
f1_ax1.add_feature(cfeature.COASTLINE.with_scale('50m'))
f1_ax1.set_xticks(np.arange(leftlon,rightlon+10,10), crs=ccrs.PlateCarree())
f1_ax1.set_yticks(np.arange(lowerlat,upperlat+10,10), crs=ccrs.PlateCarree())
f1_ax1.xaxis.set_major_formatter(lon_formatter)
f1_ax1.yaxis.set_major_formatter(lat_formatter)



C = f1_ax1.contourf(lonp, latp,ALL_p_n_P*12,levels=[-48,-24,0,24,48,72],colors=['red','tomato','lightblue','royalblue','blue'], transform=ccrs.PlateCarree())
position=fig1.add_axes([0.72, 0.171, 0.02, 0.453])
fig1.colorbar(C,cax=position,orientation='vertical')
R=f1_ax1.plot(112.823, 25.1863, 'o',color='red',  transform=ccrs.PlateCarree())

plt.savefig(r"Figure\IOD\ALL_p_n_20210301.svg",bbox_inches='tight')